In [3]:
#importing modules
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import unicodedata
from tqdm import tqdm
import gensim
import multiprocessing
import random
import pandas as pd

In [4]:
#file I need
file = ('C:\\Users\\mjjyo\\OneDrive\\Desktop\\openavenuesmicrointernship\\open_ave_data.csv')
file = pd.read_csv(file)

In [5]:
#cleaning the file. 
#dropping unneeded column
file.drop('Unnamed: 0', axis=1)
#making all lowercase
file = file.apply(lambda x: x.astype(str).str.lower())
#getting rid of labels within data
file['findings'] = file['findings'].str.replace('findings', '')
file['clinicaldata'] = file['clinicaldata'].str.replace('clinical', '')
file['clinicaldata'] = file['clinicaldata'].str.replace('indication', '')
file['clinicaldata'] = file['clinicaldata'].str.replace('history', '')
file['clinicaldata'] = file['clinicaldata'].str.replace('data', '')
file['clinicaldata'] = file['clinicaldata'].str.replace('information', '')
file['ExamName'] = file['ExamName'].str.replace('exam', '')
file['impression'] = file['impression'].str.replace('impression', '')

In [7]:
file.dropna()

,Unnamed: 0,ReportText,findings,clinicaldata,ExamName,impression
0,0,exam: chest radiography exam date: 06/01/2019 ...,: lungs/pleura: no focal opacities evident. no...,: cough. \n\n,: chest radiography date: 06/01/2019 08:30 pm...,: normal 2-view chest radiography.
1,1,exam: chest radiography exam date: 05/23/2020 ...,: lungs/pleura: no focal opacities evident. no...,: chest pain. \n\n,: chest radiography date: 05/23/2020 12:03 pm...,: no acute cardiopulmonary abnormality.
2,2,exam: chest radiography exam date: 12/13/2019 ...,: lungs/pleura: no focal opacities evident. no...,: chest pain. \n\n,: chest radiography date: 12/13/2019 03:40 pm...,: no acute cardiopulmonary process.
3,3,exam: - chest-portable history: chest pain com...,: heart size appears normal. lungs clear. \n\n,: chest pain \n\n,: - chest-portable\n\ncomparison: none,: lungs clear
4,4,exam: chest radiography exam date: 06/17/2021 ...,: lungs/pleura: no focal opacities evident. no...,": chest pain, shortness of breath. \n\n",: chest radiography date: 06/17/2021 03:09 pm...,: normal single view chest.
...,...,...,...,...,...,...
949,982,chest radiograph: pa and lateral views history...,lungs: the lung fields demonstrate normal degr...,: r05 cough \n\n,chest radiograph: pa and lateral views\n\ncomp...,: no acute pulmonary disease.
950,983,exam description: x-ray single view chest. cli...,: the lungs are well expanded and are clear. t...,": 68 years male, sob \n\n",description: x-ray single view chest.\n\ntech...,: no evidence of acute intrathoracic disease. ...
951,984,exam: chest radiography exam date: 04/02/2018 ...,: lungs/pleura: no focal opacities evident. no...,: chest pain. \n\n,: chest radiography date: 04/02/2018 10:49 pm...,: normal single view chest.
952,985,exam: xr chest 1 vw history: chest pain techni...,: lines and tubes: none cardiomediastinal: the...,: chest pain \n\n,: xr chest 1 vw\n\ntechnique: chest ap view(s)...,: no acute cardiopulmonary process dictated by...


In [9]:
#files to do vector on
exam_names = file['ExamName'].dropna().tolist()

impressions= file['impression'].dropna().tolist()

findings = file['findings'].dropna().tolist()

clinical_data = file['clinicaldata'].dropna().tolist()


In [27]:
#making corpus
corpus = exam_names + impressions + findings + clinical_data


In [28]:
def clean_data(w):
    w = w.lower()
    w=re.sub(r'[^\w\s]','',w)
    w=re.sub(r"([0-9])", r" ",w)
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words)

In [31]:
stopwords_list=stopwords.words('english')
adjusted_corpus = []
#cleaning data and making it so the corpus is a list of lists of words
for i in corpus:
    cleaned = clean_data(i)
    cleaned = cleaned.split(' ')
    adjusted_corpus.append(cleaned)


In [33]:
#word2vec model
cores= multiprocessing.cpu_count()
model = Word2Vec(min_count=5,window=5,workers=cores-1,max_vocab_size=100000)

In [34]:
model.build_vocab(adjusted_corpus)

In [43]:
model.build_vocab(adjusted_corpus, update=True)

In [46]:
model.train(adjusted_corpus, total_examples=model.corpus_count,epochs=50)

(743239, 1566350)

In [49]:
path = r"C:\Users\mjjyo\OneDrive\Desktop\openavenuesmicrointernship\word2vecmodel"

In [50]:
model.save(path)


In [51]:
trained_model = gensim.models.Word2Vec.load(path)

In [60]:
key = 'chest'
trained_model.wv.most_similar(positive=[key],topn=5)

[('sent', 0.7184100151062012),
 ('review', 0.7062097787857056),
 ('date', 0.661436140537262),
 ('ination', 0.6530739068984985),
 ('routine', 0.6444733738899231)]

In [61]:
trained_model.wv.similarity('chest', 'routine')


0.6444733

In [63]:
trained_model.wv.doesnt_match(['radiography','chest','breathing'])

'breathing'

In [68]:
trained_model.wv.most_similar(positive=
                           ['radiography', 'breath'],
                           negative=['breathing'], topn=5)

[('appa', 0.6259362101554871),
 ('routine', 0.6186726093292236),
 ('single', 0.524420440196991),
 ('lat', 0.5034644603729248),
 ('portably', 0.49046167731285095)]